In [7]:
import sys
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
# from Bio.SeqFeature import SeqFeature, FeatureLocation
# from Bio import pairwise2
# import warnings
# from Bio import BiopythonWarning
# import re
import csv



# make a dictionary of the csv file and clean up the accession numbers so it matches the viral data
#----------------------------------------------------------------------------------------------------------------
reader = csv.DictReader(open('TIS_annotation.csv'))

TIS_annotation_dict = {}

for row in reader:
    
    #get old key
    key  = row['Accession_#']
    start = key.find('NC')
    end = key.find('.', start)
    
    #strip out the verision number
    cleaned_key = key[start:end]
    
    #makes a new correct TIS annotation dictionary
    TIS_annotation_dict[cleaned_key] = row['primary_TIS_type']
#----------------------------------------------------------------------------------------------------------------



In [8]:
import sys
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
# annotate TIS with TIS type
#----------------------------------------------------------------------------------------------------------------
annotated_extracted_TIS_list = []

filename = "fixed_accession_viral_all_30up_toSTOP.fasta"

#for every record in the fasta file, delimited by >
for record in SeqIO.parse(filename, "fasta"):
    
    record_description = str(record.description)
    
    #make a new record with the sequence, name of protein, and the TIS type
    annotated_record = SeqRecord(record.seq,
                                 record.name,
                                 description = record_description + str(TIS_annotation_dict.get(record.id)))

    annotated_extracted_TIS_list.append(annotated_record)

#----------------------------------------------------------------------------------------------------------------




# IRES
# IRES, Leaky Scanning
# Leaky Scanning
# NSP3-3'UTR
# Ribosome Shunting
# Scanning
# Scanning, IRES
# Termination-Reinitiation
# VPg
# UNKNOWN



#have to separate it out into different files



# write out annotated TIS list to file
SeqIO.write(annotated_extracted_TIS_list, "annotated_" + filename, "fasta")



# def set_rna_TIS(fullpath, filename):

#     annotated_extracted_TIS_list = []

#     for record in SeqIO.parse(fullpath, "fasta"):
#         record_description = str(record.description)
#                                                                                                 #you can change this
#         annotated_record = SeqRecord(record.seq, record.name,description = record_description + "Scanning")

#         annotated_extracted_TIS_list.append(annotated_record)
        
#     SeqIO.write(annotated_extracted_TIS_list, "annotated_" + filename +".fasta", "fasta")
#     return




# creates a list of the files in this directory
# raw_datadir_listing = os.listdir(os.getcwd())




# loops over the list of files


# for files in raw_datadir_listing:
#     if files.startswith('viral_all_30up_toSTOP'):
#         full_path = os.path.join(os.getcwd(), files)
#         filename = os.path.splitext(files)[0]
#         set_viral_TIS(full_path, filename)
        

    # if files.startswith('extracted_TIS_rna'):
    #     full_path = os.path.join(os.getcwd(), files)
    #     filename = os.path.splitext(files)[0]
    #     set_rna_TIS(full_path, filename)

2528

# CODE FOR INTERACE WITH ENTREZ

In [8]:
#search ncbi GenBank with term biopython
handle = Entrez.esearch(db ="nucleotide", term = "biopython")

In [6]:
record = Entrez.read(handle)
record["IdList"]

['28011774', '24929426', '24497503', '24267035', '24194598', '23842806', '23157543', '22909249', '22399473', '21666252', '21210977', '20015970', '19811691', '19773334', '19304878', '18606172', '21585724', '16403221', '16377612', '14871861']

# Fix unnown accession number in original fasta

In [25]:
from Bio import SeqIO
from Bio import Entrez
from Bio import SeqIO

Entrez.email = "laksdjfkjaskldfjkla@gmail.com"


fixed_accession_TIS_list = []


for seq_record_local in SeqIO.parse("viral_all_30up_toSTOP.fasta", "fasta"):
    
    
    #copy all seq_record_local to new file 
    
    
    if "unknown" in seq_record_local.name:
        
        
        # get the protein name per sequence
        seq_description = seq_record_local.description

        start = seq_description.find('P_')
        end = seq_description.find('|', start)

        full_protein_name = seq_description[start-1:end]
        

        handle = Entrez.efetch(db="protein", rettype="gp", retmode="text",
                       id=full_protein_name)
        
        
        # search NCBI protein db and update all missing genome accession numbers
        
        for protein_id_remote in SeqIO.parse(handle, "genbank"):
            
            #get the accession number that links to the protein ID in the ncbi db
            accession_num_unstrip = protein_id_remote.annotations["db_source"]
            
            start = accession_num_unstrip.find('NC_')
            end = accession_num_unstrip.find('.',start)
            
            accession_num = accession_num_unstrip[start:end]
            
            
            seq_record_local.id = accession_num
            seq_record_local.name = accession_num
            seq_record_local.description = "|" +full_protein_name +"|"

            
            fixed_accession_record = SeqRecord(seq_record_local.seq,
                                               id = seq_record_local.id,
                                               name = seq_record_local.name, 
                                               description = seq_record_local.description)

            fixed_accession_TIS_list.append(fixed_accession_record)
            
#             print("changed seq record")

        handle.close()
    else:
        
        #else append the unchanged seq_record into the fixed_accession_TIS_list
        
        fixed_accession_TIS_list.append(seq_record_local)
#         print("unchanged seq record")
        
        
#have to write the fixed unknown accession number to file



SeqIO.write(fixed_accession_TIS_list, "fixed_accession_" + "viral_all_30up_toSTOP.fasta", "fasta")



    
    

In [4]:
# len(fixed)
len(fixed_accession_TIS_list)

#should have 2528 sequences

count = 0
for seq_record in SeqIO.parse("annotated_fixed_accession_viral_all_30up_toSTOP.fasta", "fasta"):
    count = count+1
    print(count)

In [21]:
#some fasta will contain duplicates b/c of sharing TIS types

Scanning = []


IRES = []

LS = []
NSP = []
RS = []

TR = []

VPG = []

UNKNOWN = []


# how many delimiters in to split
#2nd delimiter
n = 2

for seq_record in SeqIO.parse("annotated_fixed_accession_viral_all_30up_toSTOP.fasta", "fasta"):
    
    #split string after 2nd delimiter
    groups = seq_record.description.split('|')
    TIS_type = '|'.join(groups[n:])
    
    
    if TIS_type == "Scanning":
        Scanning.append(seq_record)
    elif TIS_type == "Scanning, IRES":
        Scanning.append(seq_record)
        IRES.append(seq_record)
    elif TIS_type == "IRES":
        IRES.append(seq_record)
#     elif TIS_type == "IRES, Leaky Scanning":
#         IRES.append(seq_record)
#         LS.append(seq_record)
    elif TIS_type == "Leaky Scanning":
        LS.append(seq_record)
    elif TIS_type == "NSP3-3'UTR":
        NSP.append(seq_record)
    elif TIS_type == "Ribosome Shunting":
        RS.append(seq_record)
    elif TIS_type == "Termination-Reinitiation":
        TR.append(seq_record)
    elif TIS_type == "VPg":
        VPG.append(seq_record)
    elif TIS_type == "UNKNOWN":
        UNKNOWN.append(seq_record)
    else:
        print(seq_record.name)
        print(seq_record.description)

# Scanning
# Scanning, IRES
# IRES
# IRES, Leaky Scanning
# Leaky Scanning
# NSP3-3'UTR
# Ribosome Shunting

# Termination-Reinitiation
# VPg
# UNKNOWN

NC_004108 |NP_671968.1|IRES or Leaky Scanning
NC_004109 |NP_671969.1|IRES or Leaky Scanning
NC_004110 |NP_671970.1|IRES or Leaky Scanning
NC_004110 |NP_671971.1|IRES or Leaky Scanning
NC_005218 |NP_941977.1|Scanning or IRES
NC_005219 |NP_941978.1|Scanning or IRES
NC_005222 |NP_941982.1|Scanning or IRES


In [17]:

# Scanning = []
# IRES = []
# LS = []
# NSP = []
# RS = []
# TR = []
# VPG = []
# UNKNOWN = []

SeqIO.write(Scanning, "SCANNING_" + "viral_all_30up_toSTOP_SEP.fasta", "fasta")
SeqIO.write(IRES, "IRES_" + "viral_all_30up_toSTOP_SEP.fasta", "fasta")
SeqIO.write(LS, "LEAKY_" + "viral_all_30up_toSTOP_SEP.fasta", "fasta")
SeqIO.write(NSP, "NSP_" + "viral_all_30up_toSTOP_SEP.fasta", "fasta")
SeqIO.write(RS, "RIBOSOMESHUNTING_" + "viral_all_30up_toSTOP_SEP.fasta", "fasta")
SeqIO.write(TR, "TERMINATIONREINITIATION_" + "viral_all_30up_toSTOP_SEP.fasta", "fasta")
SeqIO.write(VPG, "VPG_" + "viral_all_30up_toSTOP_SEP.fasta", "fasta")
SeqIO.write(UNKNOWN, "UNKNOWN_" + "viral_all_30up_toSTOP_SEP.fasta", "fasta")




336

In [18]:
len(Scanning)+len(IRES)+len(LS)+len(NSP)+len(RS)+len(TR)+len(VPG)+len(UNKNOWN)

# Scanning = []


# IRES = []

# LS = []
# NSP = []
# RS = []

# TR = []

# VPG = []

# UNKNOWN = []

# Scanning[0]

2521